In [1]:
#import packages
import pandas as pd
from nltk.tokenize import TweetTokenizer

## Data Cleaning (preliminary)

In [110]:
tweets = pd.read_pickle("../data/train.pkl")
tweets.head()

,tweet_id,image_id,text_info,text_info_conf,image_info,image_info_conf,text_human,text_human_conf,image_human,image_human_conf,image_damage,image_damage_conf,tweet_text,image_url,image_path,event
0,905274232590004225,901671684478029825_0,not_informative,1.0000,informative,1.0,not_relevant_or_cant_judge,1.0000,infrastructure_and_utility_damage,1.0,severe_damage,1.0000,"CONGRATS ON HITTING YOIR GOAL GUYS, I'm sure t...",http://pbs.twimg.com/media/DINhGs8XcAE1H7q.jpg,data_image/hurricane_harvey/27_8_2017/90167168...,harvey
1,901646074527535105,901646074527535105_0,informative,0.6822,informative,1.0,injured_or_dead_people,0.6822,infrastructure_and_utility_damage,1.0,severe_damage,0.6728,RT @ajwamood: #ajwamood : Harvey the first maj...,http://pbs.twimg.com/media/DILxh_xWAAAfJDY.jpg,data_image/hurricane_harvey/27_8_2017/90164607...,harvey
2,901646074527535105,901646074527535105_1,informative,0.6822,informative,1.0,injured_or_dead_people,0.6822,infrastructure_and_utility_damage,1.0,severe_damage,1.0000,RT @ajwamood: #ajwamood : Harvey the first maj...,http://pbs.twimg.com/media/DILxiisXYAAokz_.jpg,data_image/hurricane_harvey/27_8_2017/90164607...,harvey
3,901646074527535105,901646074527535105_2,informative,0.6822,informative,1.0,injured_or_dead_people,0.6822,infrastructure_and_utility_damage,1.0,severe_damage,0.6528,RT @ajwamood: #ajwamood : Harvey the first maj...,http://pbs.twimg.com/media/DILxjSUWAAEIaWH.jpg,data_image/hurricane_harvey/27_8_2017/90164607...,harvey
4,901646074527535105,901646074527535105_3,informative,0.6822,informative,1.0,injured_or_dead_people,0.6822,infrastructure_and_utility_damage,1.0,severe_damage,0.6812,RT @ajwamood: #ajwamood : Harvey the first maj...,http://pbs.twimg.com/media/DILxkWJXgAANF-E.jpg,data_image/hurricane_harvey/27_8_2017/90164607...,harvey


In [111]:
#isolating relevant columns
tweets.drop(columns=["tweet_id", "text_info_conf", "text_human_conf", "image_id", "image_info", "image_info_conf", "image_human", "image_human_conf", "image_damage", "image_damage_conf", "image_url", "image_path"], inplace=True)

In [112]:
#making text_info binary

#drop rows where text_info == "dont_know_or_cant_judge"
tweets = tweets[tweets.text_info != "dont_know_or_cant_judge"]

#map new values
tweets["info_binary"] = tweets.text_info.map({"informative":1,"not_informative":0})

In [113]:
tweets.head()

,text_info,text_human,tweet_text,event,info_binary
0,not_informative,not_relevant_or_cant_judge,"CONGRATS ON HITTING YOIR GOAL GUYS, I'm sure t...",harvey,0
1,informative,injured_or_dead_people,RT @ajwamood: #ajwamood : Harvey the first maj...,harvey,1
2,informative,injured_or_dead_people,RT @ajwamood: #ajwamood : Harvey the first maj...,harvey,1
3,informative,injured_or_dead_people,RT @ajwamood: #ajwamood : Harvey the first maj...,harvey,1
4,informative,injured_or_dead_people,RT @ajwamood: #ajwamood : Harvey the first maj...,harvey,1


In [126]:
tweets.info_binary.value_counts(1)

1    0.720242
0    0.279758
Name: info_binary, dtype: float64

In [92]:
import inflect

In [ ]:
#make tweets lowercase
lower_tweets = []
for tweet in tweets.tweet_text:
    lower_tweets.append(tweet.lower())

lower_tweets = pd.DataFrame(lower_tweets)

lower_tweets.rename(columns={0:"tweet"}, inplace=True)

tweets.tweet_text = lower_tweets.tweet

#truncate elongations (NO CODE FOR NOW)

#spell out numbers (NO CODE FOR NOW)

#replace all usernames with 'USERID'
tweets.tweet_text = tweets.tweet_text.replace(r'@\S+', 'USERID', regex=True)

#replace all URLs with HTTP
tweet.tweet_text = tweets.tweet_text.replace(r'http\S+', 'HTTP', regex=True).replace(r'www\S+', 'HTTP', regex=True)

#remove "RT" string
tweets.tweet_text = tweets.tweet_text.map(lambda x: x.lstrip('rt'))

## Preprocessing (preliminary)

In [114]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


In [121]:
from sklearn.feature_extraction.text import CountVectorizer
#Name variables 

X = tweets[["tweet_text"]]
y = tweets["info_binary"]

#train test split
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.25,
                                                    random_state=42,
                                                    stratify=y)

## Preliminary Model: CountVectorizer and Logistic Regression 

In [ ]:
#CountVectorizer 

cvec = CountVectorizer(stop_words="english")
X_train_cvec = pd.DataFrame(cvec.fit_transform(X_train['tweet_text']).todense(), columns = cvec.get_feature_names())
X_test_cvec = pd.DataFrame(cvec.transform(X_test['tweet_text']).todense(), columns = cvec.get_feature_names())
lr = LogisticRegression()
lr_model = lr.fit(X_train_cvec, y_train)
predictions = lr.predict(X_test_cvec)

In [122]:
lr.score(X_train_cvec, y_train)

0.9816640378548895

In [123]:
lr.score(X_test_cvec, y_test)

0.848314606741573

In [127]:
X_train_cvec.head()

,00,000,000gal,0044,00dgd1hm76,00gac21m45,00m3npn84t,00pm,00qztghl9v,00wksnqlp1,...,ὤfὤcἷb,ὥ1ὢ8ὥ1recent,ᾐ8,ᾑ1,ᾑ4,ᾑ7,ᾑ8,ᾓ7ἷd,ᾔa,艾玛飓风
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
